In [1]:
## Import the things we need
import time # Useful for scraping by using time.sleep(num_seconds) to force your scraper to pause
import requests # Basic library for requests
from selenium import webdriver # Getting a driver from Selenium
from bs4 import BeautifulSoup # Getting a BeautifulSoup parser
from pymongo import MongoClient # Getting a client for connecting to MongoDB

In [2]:
url = "https://www.streetadvisor.com/find-neighborhood-by-city/seattle-king-county-washington"

In [3]:
# Try using requests first
r = requests.get(url)

In [5]:
status_code = r.status_code
print status_code

if status_code == 200:
    print "Successful scrape! Status code: {}".format(status_code)
else:
    print "Error scraping! Status code: {}".format(status_code)

200
Successful scrape! Status code: 200


In [6]:
## Pull the html out of the page
html = r.content

In [7]:
soup = BeautifulSoup(html, 'html.parser')

In [8]:
divs = soup.select("div.col-group a")

In [10]:
len(divs)

74

In [11]:
all_url_ext = []
for div in divs:
    all_url_ext.append(div['href'])

In [13]:
all_url_ext[0:10]

[u'/alki-seattle-king-county-washington',
 u'/arbor-heights-seattle-king-county-washington',
 u'/atlantic-seattle-king-county-washington',
 u'/ballard-seattle-king-county-washington',
 u'/beacon-hill-seattle-king-county-washington',
 u'/belltown-seattle-king-county-washington',
 u'/briarcrest-shoreline-king-county-washington',
 u'/broadview-bitter-lake-seattle-king-county-washington',
 u'/broadway-capitol-hill-seattle-king-county-washington',
 u'/cedar-park-meadowbrook-seattle-king-county-washington']

In [14]:
root_url="https://www.streetadvisor.com"
url_list=[]
for ext in all_url_ext:
    url_list.append('https://www.streetadvisor.com{}'.format(ext))

In [16]:
url_list[0:10]

['https://www.streetadvisor.com/alki-seattle-king-county-washington',
 'https://www.streetadvisor.com/arbor-heights-seattle-king-county-washington',
 'https://www.streetadvisor.com/atlantic-seattle-king-county-washington',
 'https://www.streetadvisor.com/ballard-seattle-king-county-washington',
 'https://www.streetadvisor.com/beacon-hill-seattle-king-county-washington',
 'https://www.streetadvisor.com/belltown-seattle-king-county-washington',
 'https://www.streetadvisor.com/briarcrest-shoreline-king-county-washington',
 'https://www.streetadvisor.com/broadview-bitter-lake-seattle-king-county-washington',
 'https://www.streetadvisor.com/broadway-capitol-hill-seattle-king-county-washington',
 'https://www.streetadvisor.com/cedar-park-meadowbrook-seattle-king-county-washington']

In [17]:
reviewdict={}
for url in url_list:
    r=requests.get(url)
    reviews=[]
    if r.status_code == 200:
        html = r.content
        soup = BeautifulSoup(html, 'html.parser')
        neighborhood=soup.select("div.location-overview h1")[0].text
        divs=soup.select("div.description")
        for div in divs:
            reviews.append(div.text)
        
        reviewdict[neighborhood]=reviews
        
        client = MongoClient()
        db=client.Street_Advisor
        collection = db.neighborhood
        collection.insert_one({'name': neighborhood, 'url' : url,'date_accessed':'03-15-2017','reviews': reviews, 'raw_html':html})
        time.sleep(10)
    else:
        break